# Thymus ageing atlas: T/NK cell frequencies

In [ ]:
import os
import sys
import session_info
from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import anndata as ad
import hdf5plugin

# Add repo path to sys path (allows to access scripts and metadata from repo)
repo_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/T_NK_compartment'
sys.path.insert(1, repo_path) 
sys.path.insert(2, '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts')

# Autoreload custom scripts
%load_ext autoreload
%autoreload 2

# Define paths
plots_path = f'{repo_path}/plots/'
data_path = f'{repo_path}/data/'
model_path = os.path.join(repo_path, 'models')
general_data_path = '/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/data'

print('Dir for plots: {}'.format(plots_path))
print('Dir for data: {}'.format(data_path))

# Formatting
from matplotlib import font_manager
font_manager.fontManager.addfont("/nfs/team205/ny1/ThymusSpatialAtlas/software/Arial.ttf")
plt.style.use('/nfs/team205/lm25/thymus_projects/thymus_ageing_atlas/General_analysis/scripts/plotting/thyAgeing.mplstyle')

# Import custom scripts
from utils import get_latest_version,update_obs,freq_by_donor
from anno_levels import get_ct_levels, get_ct_palette, age_group_levels, age_group_palette
from plotting.utils import plot_grouped_boxplot, calc_figsize, thyAgeing_colors, thyAgeing_greys, get_tint_palette

# Load data


In [ ]:
# Load adata
object_version = 'v5_2025-04-03'
adata = ad.read_h5ad(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_{object_version}.zarr')

# Add new annotations to adata
ct_anno = pd.read_csv(f'{general_data_path}/objects/rna/thyAgeing_all_scvi_v4_2025-02-04_curatedAnno_v10.csv', index_col = 0)
for c in ct_anno.columns:
    if c in adata.obs.columns:
        adata.obs.drop(c, axis = 1, inplace = True)
adata.obs = adata.obs.join(ct_anno)

# Filter data (only include annotated cells)
adata = adata[~adata.obs['taa_l5'].str.contains('locnt|-sp|explore', na = True)]

# Update metadata
latest_meta_path = get_latest_version(dir = f'{general_data_path}/metadata', file_prefix='Thymus_ageing_metadata')
latest_meta = pd.read_excel(latest_meta_path)
update_obs(adata, latest_meta, on = 'index', ignore_warning = True)

adata

In [ ]:
# Define columns
col_cell_type_broad = 'taa_l3'
col_cell_type_fine = 'taa_l4'
col_cell_type_broad_levels = [c for c in get_ct_levels(col_cell_type_broad, taa_l1 = ['T', 'NK']) if c in adata.obs[col_cell_type_broad].unique().tolist()]
col_cell_type_fine_levels = [c for c in get_ct_levels(col_cell_type_fine, taa_l1 = ['T', 'NK']) if c in adata.obs[col_cell_type_fine].unique().tolist()]
col_age_group = 'age_group'
col_age_group_levels = eval(f'{col_age_group}_levels')

In [ ]:
# Create anno df
anno_df = adata.obs[['sample', 'donor', 'sex', 'sort', 'study', col_age_group, 'age_months' ,col_cell_type_broad, col_cell_type_fine, 'taa_l5']].copy()
anno_df.dropna(subset = 'taa_l5', inplace = True)

anno_df.head()

In [ ]:
anno_df['taa_l5'].isna().sum()

## CD4/CD8 ratio

In [ ]:
df = anno_df[(anno_df[col_cell_type_fine].isin(['T_CD4_naive', 'T_CD8_naive'])) & (anno_df['sort'].isin(['TOT', 'CD3P']))][[col_age_group, col_cell_type_fine, col_cell_type_broad,'donor', 'sample', 'sex']].copy()
df = df.groupby(['sample', 'donor', col_cell_type_fine]).size().to_frame(name = "n_cells").reset_index().pivot_table(values = 'n_cells', columns = col_cell_type_fine, index = ['donor', 'sample']).merge(df[['sample', 'donor']].drop_duplicates(), on = ['sample', 'donor'], how = 'inner')
df['ratio'] = df['T_CD4_naive'] / df['T_CD8_naive']
df= df.groupby('donor', observed=True).agg(ratio_mean = ('ratio', np.mean),ratio_sd = ('ratio', np.std)).reset_index().merge(adata.obs[['donor', 'age_group', 'sex', 'age_months']], on = 'donor', how = 'left')
df = df.drop_duplicates()

plt.figure(figsize=(2,1))
ax = plt.axes()
sns.scatterplot(data = df, x = 'age_months', y = 'ratio_mean', hue = 'sex', sizes = 1.5,
                hue_order = ['M', 'F'], linewidth = 0.5, 
                palette = [thyAgeing_colors['blue'], thyAgeing_colors['orange']], ax = ax)
ax.axhline(y=1, color=thyAgeing_greys['grey4'], linestyle='--')

plt.xlabel('Age [years]')
plt.ylabel('CD4/CD8 ratio')
plt.legend(title = 'Gender')
ax.set_xticklabels([int(x/12) for x in ax.get_xticks()])
ax.set_ylim(0, 4)
plt.tight_layout()
    
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_cd4cd8ratio_vs_age_lineplot.pdf')

In [ ]:
df = anno_df[(anno_df[col_cell_type_fine].isin(['T_CD4_naive', 'T_CD8_naive', 'T_Treg'])) & (anno_df['sort'].isin(['TOT', 'CD3P']))][[col_age_group, col_cell_type_fine, col_cell_type_broad,'donor', 'sample', 'sex']].copy()
df = df.groupby(['sample', 'donor', col_cell_type_fine]).size().to_frame(name = "n_cells").reset_index().pivot_table(values = 'n_cells', columns = col_cell_type_fine, index = ['donor', 'sample']).merge(df[['sample', 'donor']].drop_duplicates(), on = ['sample', 'donor'], how = 'inner')
df['ratio'] = (df['T_CD4_naive'] + df['T_Treg'])/ df['T_CD8_naive']
df = df.dropna()
df = df.groupby('donor', observed=True).agg(ratio_mean=('ratio', np.mean), ratio_sd=('ratio', np.std)).reset_index().merge(adata.obs[['donor', 'age_group', 'sex', 'age_months']].drop_duplicates(), on='donor', how='left')

plt.figure(figsize=(2,1))
ax = plt.axes()
sns.scatterplot(data = df, x = 'age_months', y = 'ratio_mean', hue = 'sex', sizes = 1.5,
                hue_order = ['M', 'F'], linewidth = 0.5, 
                palette = [thyAgeing_colors['blue'], thyAgeing_colors['orange']], ax = ax)
ax.axhline(y=1, color=thyAgeing_greys['grey4'], linestyle='--')

plt.xlabel('Age [years]')
plt.ylabel('CD4/CD8 ratio')
plt.legend(title = 'Gender')
ax.set_xticklabels([int(x/12) for x in ax.get_xticks()])
ax.set_ylim(0, 4)
plt.tight_layout()
    
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_cd4cd8ratioInclTreg_vs_age_lineplot.pdf')

## Cell frequencies

### Relative to T compartment

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT', 'CD3P'])) & (anno_df[col_cell_type_fine].isin(col_cell_type_fine_levels))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

plot_grouped_boxplot(data = df, x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = col_cell_type_fine_levels, hue_order = col_age_group_levels, 
                    palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(height = 'half', height_ratio = 0.75),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_relTfreqFine')
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_relTfreqFine_boxplot.pdf')

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT', 'CD3P'])) & (anno_df[col_cell_type_fine].isin(col_cell_type_fine_levels))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

anno_cat = {'Dev': ['T_DN(early)', 'T_DN(P)', 'T_DN(Q)', 'T_DN(late)', 'T_DP(P)', 'T_DP(Q)', 'T_αβT(entry)', 'T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg', #'B_dev', 'pDC'
                    ],
            'Mature': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc',],
            'Innate' : ['T_CD8αα(entry)', 'T_CD8αα(I)', 'T_CD8αα(II)','T_γδT','T_MAIT', 'ILC','NK_tr']}

#[c for c in col_cell_type_fine_levels if c not in [c for c in anno_cat.values() for c in c]]

for cat in anno_cat.keys():
    plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(anno_cat[cat])], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = anno_cat[cat], hue_order = col_age_group_levels, 
                        palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                        x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(width_ratio = len(anno_cat[cat])/len(col_cell_type_fine_levels),height = 60))
    plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_relTfreq{cat}_boxplot.pdf')

In [ ]:
# df = freq_by_donor(adata, sample_col = 'sample', donor_col = 'donor', summary_col=[col_age_group, col_cell_type_fine, 'age', 'sort'])
# df['donor'] = df['sort'] + '_' + df['donor'] + '_' + df['age'] 
# for a in df.age_group.unique():
#     ax = df[df.age_group == a].pivot_table(index = 'donor', columns = col_cell_type_fine, values = 'mean_prop').plot(kind = 'bar', stacked = True, figsize = (20, 5), color = sns.color_palette('tab20', n_colors = len(col_cell_type_fine_levels)))
#     plt.title(f'Age group: {a}')
#     plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#     plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine_barplot_{a}.png', dpi = 300, bbox_inches = 'tight')

### Relative to whole object

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(col_cell_type_fine_levels)], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = col_cell_type_fine_levels, hue_order = col_age_group_levels, 
                    palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(height = 55, width = 95),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine')
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine_boxplot.pdf')

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

anno_cat = {'Dev': ['T_DN(early)', 'T_DN(P)', 'T_DN(Q)', 'T_DN(late)', 'T_DP(P)', 'T_DP(Q)', 'T_αβT(entry)', 'T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg', #'B_dev', 'pDC'
                    ],
            'Mature': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc',],
            'Innate' : ['T_CD8αα(entry)', 'T_CD8αα(I)', 'T_CD8αα(II)','T_γδT','T_MAIT', 'ILC','NK_tr']}

#[c for c in col_cell_type_fine_levels if c not in [c for c in anno_cat.values() for c in c]]

for cat in anno_cat.keys():
    plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(anno_cat[cat])], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = anno_cat[cat], hue_order = col_age_group_levels, 
                        palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                        x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, 
                        figsize = calc_figsize(width_ratio = len(anno_cat[cat])/len(col_cell_type_fine_levels),height = 60))
    plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freq{cat}_boxplot.pdf')

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

anno_cat = {'Dev': ['T_DN(early)', 'T_DN(P)', 'T_DN(Q)', 'T_DN(late)', 'T_DP(P)', 'T_DP(Q)', 'T_αβT(entry)', 'T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg', #'B_dev', 'pDC'
                    ],
            'Mature': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc',],
            'Innate' : ['T_CD8αα(entry)', 'T_CD8αα(I)', 'T_CD8αα(II)','T_γδT','T_MAIT', 'ILC','NK_tr']}

plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(anno_cat['Innate'])], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = anno_cat['Innate'], hue_order = col_age_group_levels, 
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(height = 35, width = 65),
                     #save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine'
)   
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_innateSplit_freqFine_boxplot_pub.pdf')

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

anno_cat = {'Dev': ['T_DN(early)', 'T_DN(P)', 'T_DN(Q)', 'T_DN(late)', 'T_DP(P)', 'T_DP(Q)', 'T_αβT(entry)', 'T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg', 'B_dev', 'pDC'
                    ],
            'Mature': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc',],
            'Innate' : ['T_CD8αα(entry)', 'T_CD8αα(I)', 'T_CD8αα(II)','T_γδT','T_MAIT', 'ILC','NK_tr']}

plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(anno_cat['Dev'] + anno_cat['Innate'])], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = anno_cat['Dev'] + anno_cat['Innate'], hue_order = col_age_group_levels, 
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, figsize = calc_figsize(height = 50, width = 130), format_log = True,
                     ylim = (0.0001,1)
                     #save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine'
)   
plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y*100:.1f}%'))
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_devInnateSplit_freqFine_boxplot_pub.pdf')

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type_fine, add_meta = [col_age_group])

anno_cat = {'Dev': ['T_DN(early)', 'T_DN(P)', 'T_DN(Q)', 'T_DN(late)', 'T_DP(P)', 'T_DP(Q)', 'T_αβT(entry)', 'T_Treg(agonist)', 'T_CD8_naive', 'T_CD4_naive', 'T_Treg', #'B_dev', 'pDC'
                    ],
            'Mature': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc',],
            'Innate' : ['T_CD8αα(entry)', 'T_CD8αα(I)', 'T_CD8αα(II)','T_γδT','T_MAIT', 'ILC','NK_tr'],
            'Recirc': ['T_CD8_naive_recirc','T_CD8_rm','T_CD8_em', 'T_CD8_age-assoc', 'T_CD4_naive_recirc', 'T_CD4_act', 'T_CD4_h', 'T_CD4_fh', 'T_Treg_tr', 'T_Treg_recirc', 'T_MAIT', 'ILC','NK_tr', 
                       'B_naive', 'B_mem', 'B_age-associated', 'B_GC-like', 'B_plasma', 'B_plasmablast']}

plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(anno_cat['Recirc'])], x = col_cell_type_fine, y = 'mean_prop', hue = col_age_group, order = anno_cat['Recirc'], hue_order = col_age_group_levels, 
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Cell population', y_label = 'Frequency', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(height = 60, width = 130),
                     #save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_freqFine'
                     legend_kwargs = {'ncol': 1},
)  
plt.xticks(rotation=45, ha='right') 
plt.tight_layout()
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_recircSplit_freqFine_boxplot_pub.pdf')

### Stacked barplots

In [ ]:
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col='taa_l3', add_meta = [col_age_group])

df

In [ ]:
df_sub = df.loc[df['taa_l3'].isin(['T_DN(early)', 'T_DN', 'T_DP', 'T_αβT(entry)','T_CD4', 'T_CD8', 'T_Treg'])]
df_sub = df_sub.groupby([col_age_group, 'taa_l3'], observed=True).agg(mean_prop = ('mean_prop', 'mean'), sd_prop = ('mean_prop', 'std')).reset_index()
df_sub.sort_values(by = 'taa_l3')

In [ ]:
plt.figure(figsize=calc_figsize(width = 60, height = 40))
df_sub_sorted = df_sub.copy()
df_sub_sorted['taa_l3'] = pd.Categorical(df_sub_sorted['taa_l3'], categories=col_cell_type_broad_levels, ordered=True)
df_sub_sorted['age_group'] = pd.Categorical(df_sub_sorted['age_group'], categories=col_age_group_levels, ordered=True)

# Prepare data for stacked bar plot
fig, ax = plt.subplots(figsize=calc_figsize(width = 60, height = 40))
bottom = np.zeros(len(col_age_group_levels))

color_dict = {k:v for k, v in zip(['T_DN(early)', 'T_DN', 'T_DP', 'T_αβT(entry)','T_CD4', 'T_CD8', 'T_Treg'], thyAgeing_colors.values())}

for ct in color_dict.keys():
    values = []
    for age in col_age_group_levels:
        val = df_sub_sorted.loc[(df_sub_sorted['taa_l3'] == ct) & (df_sub_sorted['age_group'] == age), 'mean_prop']
        values.append(val.values[0] if len(val) > 0 else 0)
    color = color_dict.get(ct, '#cccccc')
    ax.bar(col_age_group_levels, values, bottom=bottom, label=ct, color=color, linewidth=0.25)
    bottom += np.array(values)

ax.set_xlabel('Age group')
ax.set_ylabel('Mean proportion')
ax.set_xticks(range(len(col_age_group_levels)))
ax.set_xticklabels(col_age_group_levels, rotation=0)
ax.legend(title='Cell type', bbox_to_anchor=(1.05, 1), loc='upper left')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
plt.tight_layout()
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_relTfreqFine_stackedbar.pdf')
plt.show()

In [ ]:
# Create df with cell type and compartment-level freq
anno_df_sub = anno_df[(anno_df['sort'].isin(['TOT']))].copy()
df = freq_by_donor(anno_df_sub, sample_col = 'sample', donor_col = 'donor', summary_col='taa_l3', add_meta = [col_age_group])
df_sub = df.loc[df['taa_l3'].isin(['T_DN(early)', 'T_DN', 'T_DP', 'T_αβT(entry)','T_CD4', 'T_CD8', 'T_Treg'])]

df_tot = df_sub.groupby(['donor','age_group'], observed=True).agg(mean_prop = ('mean_prop', 'sum')).reset_index()
df_tot = df_tot.groupby('age_group', observed=True).agg(mean_prop = ('mean_prop', 'mean'), sd_prop = ('mean_prop', 'std')).reset_index()

df_sub = df_sub.groupby([col_age_group, 'taa_l3'], observed=True).agg(mean_prop = ('mean_prop', 'mean'), sd_prop = ('mean_prop', 'std')).reset_index()
df_sub['rel_prop'] = df_sub.groupby('age_group', observed=True)['mean_prop'].transform(lambda x: x / x.sum())

df_sub, df_tot

In [ ]:
from typing import List, Tuple, Optional
from matplotlib.gridspec import GridSpec

def population_stacked_plot(adata: ad.AnnData, col_cell_type : str, ctypes : List[str], plt_path : Optional[str] = None, figsize : Tuple[float] = calc_figsize(width=40, height=40), sorts : List[str] = ['TOT']):

    color_dict = {k:v for k, v in zip(ctypes, thyAgeing_colors.values())}
    
    anno_df = adata.obs[['sample', 'donor', 'sex', 'sort', 'study', col_age_group, 'age_months' ,col_cell_type, 'taa_l5']].copy()
    anno_df.dropna(subset = 'taa_l5', inplace = True)

    df = freq_by_donor(anno_df[(anno_df['sort'].isin(['TOT']))], sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type, add_meta = [col_age_group])
    df_sub = df.loc[df[col_cell_type].isin(ctypes)]
    df_tot = df_sub.groupby(['donor','age_group'], observed=True).agg(mean_prop = ('mean_prop', 'sum')).reset_index()
    df_tot = df_tot.groupby('age_group', observed=True).agg(mean_prop = ('mean_prop', 'mean'), sd_prop = ('mean_prop', 'std')).reset_index()

    df = freq_by_donor(anno_df[(anno_df['sort'].isin(sorts))], sample_col = 'sample', donor_col = 'donor', summary_col=col_cell_type, add_meta = [col_age_group])
    df_sub = df.loc[df[col_cell_type].isin(ctypes)]
    df_sub = df_sub.groupby([col_age_group, col_cell_type], observed=True).agg(mean_prop = ('mean_prop', 'mean'), sd_prop = ('mean_prop', 'std')).reset_index()
    df_sub['rel_prop'] = df_sub.groupby('age_group', observed=True)['mean_prop'].transform(lambda x: x / x.sum())

    df_sub_sorted = df_sub.copy()
    df_sub_sorted[col_cell_type] = pd.Categorical(df_sub_sorted[col_cell_type], categories=ctypes, ordered=True)
    df_sub_sorted['age_group'] = pd.Categorical(df_sub_sorted['age_group'], categories=col_age_group_levels, ordered=True)

    df_tot_sorted = df_tot.copy()
    df_tot_sorted['age_group'] = pd.Categorical(df_tot_sorted['age_group'], categories=col_age_group_levels, ordered=True)
    df_tot_sorted = df_tot_sorted.sort_values('age_group')

    # Add second panel on top for total mean_prop per age_group

    fig = plt.figure(figsize=figsize)
    gs = GridSpec(2, 1, height_ratios=[1, 3], hspace=0.1)

    # Top panel: total mean_prop barplot
    ax0 = fig.add_subplot(gs[0])
    ax0.bar(
        df_tot_sorted['age_group'],
        df_tot_sorted['mean_prop'],
        yerr=df_tot_sorted['sd_prop'],
        color=thyAgeing_greys['grey3'],
        capsize=1,
        linewidth=0.25
    )
    ax0.set_ylabel('Proportion\n(of total)')
    ax0.set_xticks([])
    ax0.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    sns.despine(ax=ax0,offset=2, trim=True)
    ax0.set_xlabel('')
    ax0.set_ylim(0, 1)

    # Bottom panel: stacked barplot as before
    ax1 = fig.add_subplot(gs[1])
    bottom = np.zeros(len(col_age_group_levels))
    for ct in color_dict.keys():
        values = []
        for age in col_age_group_levels:
            val = df_sub_sorted.loc[(df_sub_sorted[col_cell_type] == ct) & (df_sub_sorted['age_group'] == age), 'rel_prop']
            values.append(val.values[0] if len(val) > 0 else 0)
        color = color_dict.get(ct, '#cccccc')
        ax1.bar(col_age_group_levels, values, bottom=bottom, label=ct, color=color, linewidth=0.25)
        bottom += np.array(values)

    ax1.set_xlabel('Age group')
    ax1.set_ylabel('Relative frequency\n(within compartment)')
    ax1.set_xticks(range(len(col_age_group_levels)))
    ax1.set_xticklabels(col_age_group_levels, rotation=0)
    ax1.legend(title='Cell type', bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)
    ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    sns.despine(ax=ax1, offset=2, trim=True)
    plt.tight_layout()
    if not plt_path is None:
        plt.savefig(plt_path)
    plt.show()

In [ ]:
from matplotlib.gridspec import GridSpec

color_dict = {k:v for k, v in zip(['T_DN(early)', 'T_DN', 'T_DP', 'T_αβT(entry)','T_CD4', 'T_CD8', 'T_Treg'], thyAgeing_colors.values())}

plt.figure(figsize=calc_figsize(width = 40, height = 40))
df_sub_sorted = df_sub.copy()
df_sub_sorted['taa_l3'] = pd.Categorical(df_sub_sorted['taa_l3'], categories=col_cell_type_broad_levels, ordered=True)
df_sub_sorted['age_group'] = pd.Categorical(df_sub_sorted['age_group'], categories=col_age_group_levels, ordered=True)

df_tot_sorted = df_tot.copy()
df_tot_sorted['age_group'] = pd.Categorical(df_tot_sorted['age_group'], categories=col_age_group_levels, ordered=True)
df_tot_sorted = df_tot_sorted.sort_values('age_group')

# Add second panel on top for total mean_prop per age_group

fig = plt.figure(figsize=calc_figsize(width=40, height=40))
gs = GridSpec(2, 1, height_ratios=[1, 3], hspace=0.1)

# Top panel: total mean_prop barplot
ax0 = fig.add_subplot(gs[0])
ax0.bar(
    df_tot_sorted['age_group'],
    df_tot_sorted['mean_prop'],
    yerr=df_tot_sorted['sd_prop'],
    color=thyAgeing_greys['grey3'],
    capsize=1,
    linewidth=0.25
)
ax0.set_ylabel('Proportion\n(of total)')
ax0.set_xticks([])
ax0.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
sns.despine(ax=ax0,offset=2, trim=True)
ax0.set_xlabel('')
ax0.set_ylim(0, 1)

# Bottom panel: stacked barplot as before
ax1 = fig.add_subplot(gs[1])
bottom = np.zeros(len(col_age_group_levels))
for ct in color_dict.keys():
    values = []
    for age in col_age_group_levels:
        val = df_sub_sorted.loc[(df_sub_sorted['taa_l3'] == ct) & (df_sub_sorted['age_group'] == age), 'rel_prop']
        values.append(val.values[0] if len(val) > 0 else 0)
    color = color_dict.get(ct, '#cccccc')
    ax1.bar(col_age_group_levels, values, bottom=bottom, label=ct, color=color, linewidth=0.25)
    bottom += np.array(values)

ax1.set_xlabel('Age group')
ax1.set_ylabel('Relative frequency\n(within compartment)')
ax1.set_xticks(range(len(col_age_group_levels)))
ax1.set_xticklabels(col_age_group_levels, rotation=0)
ax1.legend(title='Cell type', bbox_to_anchor=(1.05, 1), loc='upper left', frameon=False)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
sns.despine(ax=ax1, offset=2, trim=True)
plt.tight_layout()
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_relTfreqFine_stackedbarRel.pdf')
plt.show()

## Fate transition rates

In [ ]:
# Calculate transition rates
transitions = [('T_DN(early)', 'T_DN(P)'), ('T_DN(P)', 'T_DN(Q)'), ('T_DN(Q)', 'T_DP(P)'), ('T_DP(P)', 'T_DP(Q)'), ('T_DP(Q)', 'T_αβT(entry)'), 
               ('T_αβT(entry)', 'T_CD4_naive'), ('T_αβT(entry)', 'T_CD8_naive'), ('T_αβT(entry)', 'T_Treg')]
transitions_order = [f'{t0} -> {t1}' for t0, t1 in transitions]

trans_rates = []
for trans in transitions:
    df = anno_df[(anno_df[col_cell_type_fine].isin(trans)) & (anno_df['sort'].isin(['TOT', 'CD3P']))]
    
    pairwise_combinations = pd.MultiIndex.from_product([df['sample'].unique(), df[col_cell_type_fine].unique()], names=['sample', col_cell_type_fine]).to_frame(index=False)
    pairwise_combinations = pairwise_combinations.merge(df[['sample','donor', col_age_group]], on = 'sample', how = 'left')
    
    df = df.groupby([col_age_group, col_cell_type_fine, 'sample', 'donor'], observed = True).size().reset_index(name='counts')
    df = df.merge(pairwise_combinations, on = [col_cell_type_fine, col_age_group, 'sample', 'donor'], how = 'outer').fillna(0)
    df = df.pivot_table(index=[col_age_group, 'sample', 'donor'], columns=col_cell_type_fine, values='counts', aggfunc='sum').fillna(0).reset_index()

    df['ratio'] = np.where(df[trans[0]] == 0, np.nan, (df[trans[1]] / df[trans[0]]) + 0.001)
    df['transition'] = f'{trans[0]} -> {trans[1]}'
    
    df = df.groupby([col_age_group, 'transition', 'donor'], observed=True).agg(ratio = ('ratio', 'mean')).reset_index()
    
    trans_rates.append(df)
    
trans_rates = pd.concat(trans_rates)

trans_rates.head()

In [ ]:
plot_grouped_boxplot(data = trans_rates.reset_index(), x = 'transition', y = 'ratio', hue = col_age_group, order = transitions_order, hue_order = col_age_group_levels, 
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Transition', y_label = 'Transition rate', legend_title = 'Age group', add_stats = True, format_log = True, figsize = calc_figsize(height = 70, width = 73), 
                     y_intercept = 1.0, ylim = (0.1, 1000),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitions',
                     legend_kwargs = {'bbox_to_anchor':(1.05, 1), 'loc':'upper left'})
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitions_boxplot.pdf')

In [ ]:
# Publication plot
trans_rates['transition'] = trans_rates['transition'].str.replace('T_', '').str.replace('->', '\n->') # Save space in plot
transitions_order = [t.replace('T_', '').replace('->', '\n->') for t in transitions_order]
p = plot_grouped_boxplot(data = trans_rates.reset_index(), x = 'transition', y = 'ratio', hue = col_age_group, order = transitions_order, hue_order = col_age_group_levels, 
                         palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Transition', y_label = 'Transition rate', legend_title = 'Age group', add_stats = True, format_log = True, figsize = calc_figsize(height = 45, width = 90), 
                     y_intercept = 1.0, ylim = (0.1,1000), legend_kwargs = {'ncol': 2},
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitions')
#plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitions_boxplot_pub.pdf')

In [ ]:
df = trans_rates.loc[trans_rates['transition'].isin(['T_αβT(entry) -> T_CD4_naive', 'T_αβT(entry) -> T_CD8_naive', 'T_αβT(entry) -> T_Treg'])].reset_index(drop = True)
df['transition'] = df['transition'].str.replace('T_αβT(entry) ', '') # Save space in plot

plot_grouped_boxplot(data = df, 
                     x = col_age_group, y = 'ratio', hue = 'transition', order = col_age_group_levels, hue_order = ['-> T_CD4_naive', '-> T_CD8_naive', '-> T_Treg'], 
                     palette = get_tint_palette(thyAgeing_colors['yellow']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Transition', y_label = 'Transition rate', legend_title = 'SP fate', add_stats = True, format_log = True, figsize = calc_figsize(width = 50, height = 50), y_intercept = 1.0, ylim = (0.1, 100),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitionsMatureComp', remove_legend = False,
                     legend_kwargs = {'loc' : 'upper right', 'bbox_to_anchor': (1.3, 1.3), 'ncols' : 1})
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitionsMatureComp_boxplot.pdf')

In [ ]:
trans_rates

In [ ]:
df = trans_rates.loc[trans_rates['transition'].isin(['T_αβT(entry) -> T_CD4_naive', 'T_αβT(entry) -> T_CD8_naive', 'T_αβT(entry) -> T_Treg'])].reset_index(drop = True)
df.loc[df['transition'] =='T_αβT(entry) -> T_Treg', 'transition'] = 'T_αβT(entry) -> T_CD4_naive' # Save space in plot
df = df.groupby(['donor', 'transition', col_age_group], observed=True).agg(ratio = ('ratio', 'mean')).reset_index()
df['transition'] = df['transition'].str.replace('T_αβT(entry) ', '') # Save space in plot

plot_grouped_boxplot(data = df, 
                     x = col_age_group, y = 'ratio', hue = 'transition', order = col_age_group_levels, hue_order = ['-> T_CD4_naive', '-> T_CD8_naive'], 
                     palette = get_tint_palette(thyAgeing_colors['yellow']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     x_label = 'Transition', y_label = 'Transition rate', legend_title = 'SP fate', add_stats = True, format_log = True, figsize = calc_figsize(width = 50, height = 50), y_intercept = 1.0, ylim = (0.1, 100),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitionsMatureComp', remove_legend = False,
                     legend_kwargs = {'loc' : 'upper right', 'bbox_to_anchor': (1.3, 1.3), 'ncols' : 1})
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_fateTransitionsMatureComp_woTreg_boxplot.pdf')

## Absolute numbers

In [ ]:
df = anno_df[(anno_df['sort'].isin(['TOT'])) & (anno_df[col_cell_type_fine].isin(col_cell_type_fine_levels))].copy()
df = df.groupby(['sample', 'donor', col_age_group, col_cell_type_fine], observed=True).size().to_frame(name = "n_cells").reset_index()

pairwise_combinations = pd.MultiIndex.from_product([df['sample'].unique(), df[col_cell_type_fine].unique()], names=['sample', col_cell_type_fine]).to_frame(index=False)
pairwise_combinations = pairwise_combinations.merge(df[['sample','donor', col_age_group]], on = 'sample', how = 'left')
    
df = df.merge(pairwise_combinations, on = [col_cell_type_fine, col_age_group, 'sample', 'donor'], how = 'outer').fillna(0)
df = df.groupby([col_age_group, col_cell_type_fine, 'donor'], observed = True).agg(n_cells = ('n_cells', 'mean')).reset_index()

df.head()

In [ ]:
plot_grouped_boxplot(data = df, x = col_cell_type_fine, y = 'n_cells', hue = col_age_group, order = col_cell_type_fine_levels, hue_order = col_age_group_levels, 
                     x_label = 'Cell population', y_label = 'Mean number of cells', legend_title = 'Age group', add_stats = True, format_log = True, figsize = (15,7), ylim = (0.1, 10000),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_nFine')
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_nFine_boxplot.pdf')

In [ ]:
plot_grouped_boxplot(data = df.loc[df[col_cell_type_fine].isin(['T_CD8_naive', 'T_CD4_naive', 'T_Treg'])], x = col_cell_type_fine, y = 'n_cells', hue = col_age_group, order = ['T_CD8_naive', 'T_CD4_naive', 'T_Treg'], hue_order = col_age_group_levels, 
                     x_label = 'Cell population', y_label = 'Mean number of cells', legend_title = 'Age group', add_stats = True, format_log = True, figsize = (4,5), ylim = (1, 10000),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_nFineMature')
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_nFineMature_boxplot.pdf')

## Spatial distribution

In [ ]:
nhood_enrichment = pd.read_csv(f'{data_path}/objects/rna/thyAgeing_tSplitxTissue_scvi_v1_2024-11-29_nhoodEnrichment.csv', index_col=0)
nhood_enrichment.replace(pd.NA, 'none', inplace = True)
nhood_enrichment = nhood_enrichment.loc[nhood_enrichment.index.str.endswith('-0')]
nhood_enrichment.index = nhood_enrichment.index.str.replace('-0', '')
anno_df = anno_df.join(nhood_enrichment)

In [ ]:
nhood_enrichment['nhood_enrichment'].value_counts() 

In [ ]:
df_location = anno_df[(anno_df['sort'].isin(['TOT', 'CD3P'])) & (anno_df[col_cell_type_fine].isin(col_cell_type_fine_levels))].groupby(['donor', col_cell_type_fine])['nhood_enrichment'].value_counts(normalize=True).to_frame('freq').reset_index()

import seaborn as sns

import matplotlib.pyplot as plt

# Convert 'freq' to percentage
df_location['freq'] = df_location['freq']

# Plot grouped barplot
plt.figure(figsize=calc_figsize(width_ratio = 1, height_ratio = 0.5))
ax = sns.barplot(data=df_location, x=col_cell_type_fine, y='freq', hue='nhood_enrichment', order = [l for l in col_cell_type_fine_levels if l in df_location[col_cell_type_fine].unique()], 
            hue_order = ['blood', 'tissue', 'none'], ci='sd', palette = get_tint_palette(thyAgeing_colors['magenta']), linewidth = 0)
ax.set_xlabel('Cell population')
ax.set_ylabel('Proportion')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{x:.0%}'))
ax.tick_params(axis='x', rotation=90)
ax.legend(title='Location enrichment', loc='upper right', bbox_to_anchor=(1.25, 1))
plt.tight_layout()
ax.set_ylim(0, 1.0)
sns.despine(trim = True, offset = 2)
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_cellType_by_nhood.pdf')
plt.show()

In [ ]:
blood_enriched = anno_df[(anno_df['sort'].isin(['TOT', 'CD3P'])) & (anno_df['nhood_enrichment'] == 'blood')].groupby(['donor', col_age_group])[col_cell_type_fine].value_counts(normalize=True).to_frame('freq').reset_index().dropna()
plot_grouped_boxplot(data = blood_enriched, x = col_cell_type_fine, y = 'freq', hue = col_age_group, order = [c for c in col_cell_type_fine_levels if c in blood_enriched[col_cell_type_fine].unique()], hue_order = col_age_group_levels, 
                     x_label = 'Cell population', y_label = 'Proportion', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(width = 'single', height_ratio = 0.5),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_bloodPropCells',
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     )
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_bloodPropCells_boxplot.pdf')

In [ ]:
tissue_enriched = anno_df[(anno_df['sort'].isin(['TOT', 'CD3P'])) & (anno_df['nhood_enrichment'] == 'tissue')].groupby(['donor', col_age_group])[col_cell_type_fine].value_counts(normalize=True).to_frame('freq').reset_index().dropna()
plot_grouped_boxplot(data = tissue_enriched, x = col_cell_type_fine, y = 'freq', hue = col_age_group, order = [c for c in col_cell_type_fine_levels if c in tissue_enriched[col_cell_type_fine].unique()], hue_order = col_age_group_levels, 
                     x_label = 'Cell population', y_label = 'Proportion', legend_title = 'Age group', add_stats = True, format_percent = True, figsize = calc_figsize(width = 'single', height_ratio = 0.5),
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_tissuePropCells',
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     )
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_tissuePropCells_boxplot.pdf')

In [ ]:
df = tissue_enriched.merge(blood_enriched, on = ['donor', col_age_group, col_cell_type_fine], suffixes = ('_tissue', '_blood'))
df['ratio'] = df['freq_tissue'] / df['freq_blood']
df.dropna(inplace = True)  
df = df.loc[~df['taa_l4'].isin(['T_CD8αα(entry)', # Exclude innate
 'T_CD8αα(I)',
 'T_CD8αα(II)',
 'T_MAIT',
 'ILC',
 'T_γδT',
 'NK_tr',
 'T_Treg_tr',])] 
df

In [ ]:
plot_grouped_boxplot(data = df, x = col_cell_type_fine, y = 'ratio', hue = col_age_group, order = [c for c in col_cell_type_fine_levels if c in df[col_cell_type_fine].unique()], hue_order = col_age_group_levels, 
                     x_label = 'Cell population', y_label = 'Ratio [tissue/blood]', legend_title = 'Age group', add_stats = True, format_log = True, figsize = calc_figsize(width = 'single', height_ratio = 0.5), ylim = (0.01,100), y_intercept = 1,
                     save_stats = f'{data_path}/analyses/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_ratioTissueBlood',
                     palette = get_tint_palette(thyAgeing_colors['magenta']),annotator_args = dict(line_offset = 20, text_offset = -2, line_offset_to_group = 10),
                     )
plt.savefig(f'{plots_path}/freqAnalysis/all_curatedAnno_v10/thyAgeing_tSplit_ratioTissueBlood_boxplot.pdf')

## Follicular T cells

In [ ]:
anno_df

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
t_gc_goi = {'LZ_entry': ['CCR7', 'CXCR5'],
            'SLAM_interaction': ['SLAMF1', 'LY9', 'CD244', 'SLAMF7', 'SLAMF6', 'CD84'],
            'LFA1_activation': ['ITGB2', 'ITGA4', 'SEMA4B'], 
            'BCR_activation': ['CD40LG', 'ICOS', 'PDCD1'],
            'Inflammatory': ['IFITM2', 'TNFSF14', 'CXCR3', 'ITGB1', 'CXCL13', 'TOX', 'TOX2', 'IL21', 'CCL5', 'IFIT1'],
            'interleukin_signalling' : ['IL21', 'IL4', 'IL6', 'IL10'],
            'CD8' : ['CD8A', 'CD8B'],}

sc.pl.DotPlot(adata[adata.obs['taa_l1'] == 'T'], 
              groupby=col_cell_type_fine,
              var_names=t_gc_goi,
              mean_only_expressed=True,
              cmap = 'viridis',).add_totals().show()#savefig(f'{plots_path}/preprocessing/markers/tri21Thymus_all_scvi_{object_version}_{k}Markers_dotplot.png', dpi=300, bbox_inches='tight')

In [ ]:
session_info.show()